### Instalação das dependências do Projeto

- Scrapy para fazer parsear o html da pagina do 156 e coletar todas as urls dos csvs

- Duckdb para para conseguir ler os arquivos de forma otimizada

- Pyarrow para conseguir salvar os arquivos como parquet

In [1]:
!pip install -q scrapy
!pip install -q duckdb
!pip install -q pyarrow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import requests
from scrapy import Selector
from urllib.parse import urljoin
import pandas as pd
import requests
from glob import glob
from IPython.display import clear_output
import os
#import duckdb

### Scraping:

A Etapa abaixo, realiza a raspagem do site da prefeitura, coletando todas as urls dos CSVs

In [4]:
def fetch_csv_urls():
    url = 'http://dados.prefeitura.sp.gov.br/dataset/dados-do-sp156'
    response = requests.get(url)
    print(response.status_code)
    sel = Selector(text=response.text)
    csv_links = sel.css('a::attr(href)').re(r'.*\.csv$')
    csv_links = [urljoin(response.url, link) for link in csv_links]
    return csv_links

if __name__ == "__main__":
    csv_urls = fetch_csv_urls()
    for url in csv_urls:
        print(url)
        with open('urls.txt', 'a') as file:
            file.write(url + '\n')


200
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/3c12fe0a-ec4f-4a20-a987-dd177330f4ce/download/arquivofinal3tride2024.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/2d020379-fa40-4dc0-8d17-add74b119550/download/arquivofinal2tri2024.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/787c054d-3e77-46c8-8713-e4e26eb2dd55/download/arquivofinal1tride2024.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/33d41278-02de-417a-a99a-5e01d3d87952/download/arquivofinal4tride2023.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/b01a9038-969e-4630-bd3a-4abe188b7259/download/arquivofinal3tri2023.csv
http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/7978d3ba-d379-44db-993b-61b9ff406435/download/arquivofinal2tride2023.csv
http://dados.prefeitura.sp.gov.br/

### Download dos arquivos CSV

Neste momento iremos realizar o download de todos os arquivos do tipo CSV e vamos converter para parquet, que é um formato mais otimizado.

In [8]:
# Função para baixar o CSV
def download_csv(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Erro ao baixar {url}: Status {response.status_code}")


# Função para processar as URLs
def process_urls(file_with_urls):
    # Lê as URLs do arquivo de texto
    with open(file_with_urls, 'r') as file:
        urls = file.read().splitlines()

    for url in urls:
        file_name = os.path.basename(url)
        csv_path = f"../downloads/{file_name}"
        download_csv(url, csv_path)
        print(f"download {file_name}")

file_with_urls = 'urls.txt'
os.makedirs('../downloads', exist_ok=True)
process_urls(file_with_urls)


Erro ao baixar http://dados.prefeitura.sp.gov.br/dataset/0aecfa2b-aa3a-40d4-8183-0d4351b7fd0a/resource/a0f741c5-f406-4bda-b0c7-cab1af6bbd89/download/arquivofinal2tri2021.csv: Status 404
download arquivofinal2tri2021.csv


### Conversão de arquivos em CSV para PARQUET e particionamento

Nesta etapa, estamos realizando a conversão de todos os arquivos tipo CSV para parquet, que é um format otimado, e também estamos particionando os dados, para tornar as consultas mais leves.

In [17]:
import re
import unicodedata
import pandas as pd
import os
from glob import glob
from IPython.display import clear_output

def convert_columns_to_snake_case(df):
    """Converte os nomes das colunas para snake_case removendo acentos e substituindo caracteres especiais."""
    def remove_accents(input_str):
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])
    
    df.columns = [re.sub(r'[\s\-/]', '_', remove_accents(col)).lower() for col in df.columns]
    return df


def csv_to_parquet():
    for path in glob('../downloads/*.csv'):
        print(path)
        df = pd.read_csv(path, encoding='latin-1', sep=';')
        
        df = convert_columns_to_snake_case(df)

        default_values = {
            'data_de_abertura': '0000-00-00 00:00:00',
            'canal': '',
            'tema': '',
            'assunto': '',
            'servico': '',
            'logradouro': '',
            'cep': '',
            'numero': '',
            'subprefeitura': '',
            'distrito': '',
            'latitude': 0.0,
            'longitude': 0.0,
            'data_do_parecer': '0000-00-00 00:00:00',
            'status_da_solicitacao': '',
            'orgao': '',
            'nivel': '',
            'data': '0000-00-00',
            'prazo_atendimento': '',
            'qualidade_atendimento': '',
            'atendeu_solicitacao': ''
        }

        for col, default_value in default_values.items():
            if col not in df.columns:
                df[col] = default_value

        datetime_cols = ['data_de_abertura', 'data_do_parecer']
        for col in datetime_cols:
            df[col] = pd.to_datetime(df[col], errors='coerce')

        string_cols = ['numero', 'orgao', 'nivel', 'atendeu_solicitacao', 'cep']
        for col in string_cols:
            df[col] = df[col].astype(str).fillna('')

        numeric_cols = ['latitude', 'longitude']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0).astype(float)

        df['ano'] = df['data_de_abertura'].dt.year
        df['mes'] = df['data_de_abertura'].dt.month

        parquet_path = os.path.join('../database', os.path.basename(path).replace('.csv', ''))
        os.makedirs(parquet_path, exist_ok=True)
        df.to_parquet(parquet_path, index=False, partition_cols=['ano', 'mes'])

        clear_output(wait=True)

csv_to_parquet()

../downloads/arquivofinal3tri2016.csv


### Testes com Duckdb

In [21]:
import duckdb
query = """
SELECT 
  distinct status_da_solicitacao
FROM read_parquet('../database/*/*/*/*.parquet') 
"""
duckdb.sql(query).show()


┌───────────────────────┐
│ status_da_solicitacao │
│        varchar        │
├───────────────────────┤
│ ABERTA                │
│ CANCELADA             │
│ EM ANDAMENTO          │
│ AGUARDANDO APROVAÇÃO  │
│ RASCUNHO              │
│ FINALIZADA            │
│                       │
│ RELACIONADA           │
└───────────────────────┘



In [13]:
import glob
for path in glob.glob('../downloads/*'):print(path)

../downloads/arquivofinal1tri2015-copia.csv
../downloads/arquivofinal1bi2020.csv
../downloads/arquivofinal1sem2012.xlsx.csv
../downloads/arquivofinal3tri2021.csv
../downloads/arquivofinal5bi2020.csv
../downloads/arquivofinal4tri2017.csv
../downloads/arquivofinal1tri2024.csv
../downloads/arquivofinal2tri2024.csv
../downloads/arquivofinal4tri2023.csv
../downloads/arquivofinal2sem2014.xlsx.csv
../downloads/arquivofinal4tri2015-copia.csv
../downloads/arquivofinal3tride2024.csv
../downloads/arquivofinal4tri2021.csv
../downloads/arquivofinal2tri2017.csv
../downloads/arquivofinal2tri2022.csv
../downloads/arquivofinal3tri2022.csv
../downloads/arquivofinal2tri2016.csv
../downloads/arquivofinal3tri2015-copia.csv
../downloads/arquivofinal2tri2015-copia.csv
../downloads/arquivofinal2sem2012.xlsx.csv
../downloads/arquivofinal4tri2022.csv
../downloads/arquivofinal1sem2014.xlsx.csv
../downloads/arquivofinal1tri2021.csv
../downloads/arquivofinal4bi2020.csv
../downloads/arquivofinal1tri2016.csv
../down